In [33]:
import twitter
import os
import yaml
import re
import time
import tweepy
import pandas as pd
from textblob import TextBlob
from collections import Counter
import pickle

In [2]:
credentials = yaml.load(open(os.path.expanduser('~/.ssh/api_credentials.yml')))

# Try Tweepy

In [65]:
#!/usr/bin/env python
# encoding: utf-8

import tweepy #https://github.com/tweepy/tweepy
import csv

#Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""


def get_all_tweets(screen_name, api):
	"""Download the last 3240 tweets from a user. Do text processign to remove URLs and the retweets from a user.
	Adapted from https://gist.github.com/yanofsky/5436496"""
	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:

		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
	print(f'Finished getting tweets for {screen_name}')
	cleaned_text = [re.sub(r'http[s]?:\/\/.*[\W]*', '', i.text, flags=re.MULTILINE) for i in alltweets] # remove urls
	cleaned_text = [re.sub(r'@[\w]*', '', i, flags=re.MULTILINE) for i in cleaned_text] # remove the @twitter mentions 
	cleaned_text = [re.sub(r'RT.*','', i, flags=re.MULTILINE) for i in cleaned_text] # delete the retweets
	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str, tweet.created_at, cleaned_text[idx]] for idx,tweet in enumerate(alltweets)]
	
	
	return pd.DataFrame(outtweets,columns=["id","created_at","text"])


In [66]:
	auth = tweepy.OAuthHandler(credentials['twitter']['consumer_key'], credentials['twitter']['consumer_secret'],)
	auth.set_access_token(credentials['twitter']['token'], credentials['twitter']['token_secret'])
	api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [67]:
tweets_df = get_all_tweets('ericries', api)

Finished getting tweets for ericries


In [48]:
def scrub_text(tweets_df, return_bag_of_words = False, num_chars = 10):
    """Takes in a tweets DF and returns a list of sentences .
    Also creates the bag of words from the text to be used at a later time.
    num_chars specifies the minimum number of characters per sentence"""
    bag_of_words = None
    all_sentences = []
    for row in tweets_df.iterrows():
        blob = TextBlob(row[1]['text'])
        blob = blob.lower()
        blobl = blob.strip()# remove whitespace
        for sent in blob.sentences: ## append each sentence
            if len(sent) < num_chars: # sentences need to have at least ten characters
                pass
            else:
                all_sentences.append(str(sent)+" ")
        tokens = blob.tokenize()
        if bag_of_words == None:
            bag_of_words =  Counter(tokens)
        else:
            words = Counter(tokens)
            for k,v in words.items():
                if k in bag_of_words:
                    bag_of_words[k]+=v
                else:
                    bag_of_words[k]=v
    if return_bag_of_words == True:
        return all_sentences, bag_of_words
    else:
        return all_sentences
                
    

In [69]:
vc1_pitchbook_df = pd.read_csv( "../data/processed/PitchBook_CA_VCInvest=1.csv")

In [71]:
vc1_pitchbook_df[vc1_pitchbook_df['Primary Contact']=='Michael Leonard']

,Unnamed: 0,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Active Investors,HQ Location,...,Last Financing Date,Last Financing Size,Last Financing Deal Type 2,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised,VC_invested,Twitter_Username
170,978,57570-94,Developer of non-narcotic analgesic drug produ...,Trigemina,94556,Drug Discovery,Michael Leonard,2006.0,"Arrow Hedge Partners, Aurus Gestion de Inversi...","Moraga, CA",...,2013-06-17,6.0,NaN,35.0,1135.0,1098.0,12.0,7.61,1,MichaelCLeonard


In [58]:
username_errors = []
# get text from the Twitter handle of these founder
for idx,row in enumerate(vc1_pitchbook_df.iterrows()):
    total_rows = len(vc1_pitchbook_df)
    
    founder = row[1]['Primary Contact']
    company = row[1]['Company Name']
    twitter_username = row[1]['Twitter_Username']
    try:
        tweets = get_all_tweets(twitter_username )
        scrubbed_tweets = scrub_text(tweets)
        with open(f"../data/raw/founders_tweets/vc_invest=1/{company}-{founder}-{twitter_username}", "wb") as output_file:
            pickle.dump(scrubbed_tweets, output_file, protocol=pickle.HIGHEST_PROTOCOL)
    except: # not authorized to see this user's timeline
        username_errors.append(founder) ## eventually drop these usernames

    print(f"{idx/total_rows:%} percent finished")
    
 

Finished getting tweets for mgold54
0.000000% percent finished
Finished getting tweets for sbaksh
0.546448% percent finished
Finished getting tweets for notEricMin
1.092896% percent finished
Finished getting tweets for sshadfar
1.639344% percent finished
Finished getting tweets for anthemos
2.185792% percent finished
Finished getting tweets for JuliaRivka
2.732240% percent finished
Finished getting tweets for _davidcarter
3.278689% percent finished
Finished getting tweets for Katmann45
3.825137% percent finished
Finished getting tweets for bigolevy
4.371585% percent finished
Finished getting tweets for ksteckel
4.918033% percent finished
Finished getting tweets for christophroar
5.464481% percent finished
Finished getting tweets for mmJonathantf
6.010929% percent finished
Finished getting tweets for staceyepstein
6.557377% percent finished
Finished getting tweets for ChardonnayDon
7.103825% percent finished
Finished getting tweets for TimReilly316
7.650273% percent finished
Finished ge

68.852459% percent finished
Finished getting tweets for ShaverLakeRE
69.398907% percent finished
Finished getting tweets for eythorbender
69.945355% percent finished
Finished getting tweets for brucebrownfilms
70.491803% percent finished
Finished getting tweets for ThomasMunchLaur
71.038251% percent finished
Finished getting tweets for iamkunala
71.584699% percent finished
Finished getting tweets for CrisLehman
72.131148% percent finished
Finished getting tweets for Robert_CFTL
72.677596% percent finished
Finished getting tweets for robertshomebrew
73.224044% percent finished
Finished getting tweets for JasonNovi
73.770492% percent finished
Finished getting tweets for ShawnGuan29
74.316940% percent finished
Finished getting tweets for santosh79
74.863388% percent finished
Finished getting tweets for SebastianThrun
75.409836% percent finished
Finished getting tweets for appletree
75.956284% percent finished
Finished getting tweets for mrkmcnally
76.502732% percent finished
Finished gett

In [59]:
username_errors

['Robert Beaver',
 'Jing Nie',
 'Adam Neumann',
 'Oliver Tan',
 'Melissa Liu',
 'Jonathan Tang',
 'Tobin Fisher',
 'Benjamin Cohen',
 'Stefan Ryser']

In [39]:
scrubbed_tweets 

['#flyboy #flygirl victory party. ',
 'a great day at #im703wisconsin with the most amazing training… ',
 ' beyond amazing shot! ',
 ' ready for the mud fest!!! ',
 ' thank you for your leadership and counsel in helping  become the great president he is. ',
 'our team members continue to do great things in our communities! ',
 'senior developer/workflow automation - plymouth, mn - tcf bank ',
 'great for mlb! ',
 '#temporarycubsfan ',
 'mission accomplished. ',
 '#avoidthe8run8th #gocubsgo ',
 '#avoidthe8run8th #gocubsgo ',
 'unbelievable recovery run from the  marathon... the fall colors were amazing and the… ',
 'the stage is set for the 2016 twin cities marathon! ',
 'one more sleep! ',
 'this will be my 4th marathon… ',
 '\n\n#presidentialdebate #debate ',
 ' first look at our race day forecast is out! ',
 "if this delivers as promised,  we're in for a great day! ",
 'tcf introduces chip debit card with contactless payment capability ',
 'still the one: peterson keeps a diversified

# Python -Twitter

In [124]:
text = api.GetUserTimeline(screen_name='ashady',count =300, include_rts=False)

In [125]:
cleaned_text = [re.sub(r'http[s]?:\/\/.*[\W]*', '', i.text, flags=re.MULTILINE) for i in text] # remove the urls

In [127]:
cleaned_text = [re.sub(r'@[\w]*', '', i, flags=re.MULTILINE) for i in cleaned_text] # remove the @twitter mentions

In [128]:
cleaned_text

[" Nice work Eric. Love the analysis. We're building the same using AI. Starting making picks this month  demo day",
 'Flying  and really impressed by the crew. Great service and super friendly!',
 'I just published “Which 500 Startups Demo Day companies does our AI algorithm like?” ',
 'When will AI replace venture capitalists? ',
 'Venture Capital: Are any VC firms using machine learning or AI to help in deal sourcing? ',
 'NYU Stern School of Business  featuring  Stern pride goes both ways :) \n',
 'What are the common traits of successful entrepreneurs? ',
 '6 Traits of Successful Founders ',
 '    thanks Chris!! Means a lot that you like it!',
 'I just published “The Personalities of Successful Founders” ',
 ' ',
 'I just donated to  to help children in need. You can help too!: ',
 ' which one?',
 'Attending a cool session on NLP and machine learning at  ',
 'Complex #DataViz by  on  untangles   business ties ',
 ' yes photos pls!',
 'Just donated ',
 'What it means to stand again

In [129]:
api.GetFollowers(screen_name='ashady')

[User(ID=59840147, ScreenName=kvasnic),
 User(ID=868973790386151424, ScreenName=EsraaEl51255104),
 User(ID=288497184, ScreenName=JTattersfield),
 User(ID=870175789546958848, ScreenName=ADo7a93),
 User(ID=24961055, ScreenName=mattbiehl),
 User(ID=744150240136044544, ScreenName=djkingassassin),
 User(ID=711273343396278272, ScreenName=JaydaBF),
 User(ID=351100725, ScreenName=Dylan_Bostic),
 User(ID=864015366946775040, ScreenName=RehamMohamedAh7),
 User(ID=296320946, ScreenName=SAPTradeGeek),
 User(ID=866383033292771328, ScreenName=TurkiBinA2),
 User(ID=865124920598179840, ScreenName=PeaceHackMENA),
 User(ID=799012517129269248, ScreenName=MuhammedEzzat10),
 User(ID=1879978364, ScreenName=ryanhintze),
 User(ID=805559119566565376, ScreenName=ask1angel),
 User(ID=3784808057, ScreenName=bouigoudal),
 User(ID=25217355, ScreenName=_moustafa),
 User(ID=266420236, ScreenName=treyptrsn),
 User(ID=2707473229, ScreenName=DreamsdotBuild),
 User(ID=556259902, ScreenName=OmarAbdElRhmaan),
 User(ID=86024

In [108]:
def twitter_text(username):
    """Return the last 200 tweets' text from the given username"""
    text = api.GetUserTimeline(screen_name=username,count =300, include_rts=False)
    cleaned_text = [re.sub(r'http[s]?:\/\/.*[\W]*', '', i.text, flags=re.MULTILINE) for i in text] # remove the urls
    cleaned_text = [re.sub(r'@[\w]*', '', i, flags=re.MULTILINE) for i in cleaned_text] # remove the @twitter mentions
    return cleaned_text

In [130]:
twitter_text('olivercameron')

['It took ~4 years but they did it! ',
 'Surely radar and cameras is preferable to LIDAR? Primarily for range &amp; cost: ',
 '   Definitely!',
 '  did all the good work there 👌',
 'We recently rebuilt our entire  controls stack for silky smooth rides. Lots of related good stuff in here! ',
 ' Great decision!  and the team are building a world-changing product.',
 'Awesome opportunity. ',
 '😱 ',
 '   You can in the MKZ, not in the Fusion.',
 'A template for building a great company, courtesy of . ',
 ' Pumped to see what you build next! Just my personal opinion, but I would try to cure cancer with dinosaurs.',
 '😶  ',
 '    ',
 '   is the exact opposite, prefers Antarctic winds in the car.',
 'To be clear: this is a bug in the  app, not the actual car temperature ☀️ ',
 '🤔 ',
 'Insane growth! ',
 'Fun visit to  earlier. Mapping (and all that comes with it) is a huge pain-point in autonomous vehicles… ',
 'Awesome interview. ',
 '      Can you imagine?',
 'I should mention, we plan to o

In [ ]:
def twitter_followers(username):
    """REturn the number of followers for the given user. Need to sleep to not exceed the rate limit."""
    for num in range(1_000_000):
        if num %500 ==0:
            time.sleep(5)
            